In [ ]:
# save_direct_to_minio.py
import pandas as pd

df = pd.DataFrame({"a":[3,3,3], "b":["x","y","z"]})

df.to_parquet(
    "s3://mlflow/raw/df.parquet",
    index=False,
    storage_options={
        "key": "minio",
        "secret": "minio12345",
        "client_kwargs": {"endpoint_url": "http://localhost:9002"}
    },
)
print("✅ wrote to s3://mlflow/raw/df.parquet")


In [ ]:
import subprocess
subprocess.run(["dvc", "update", "dvc_meta/df.parquet.dvc"], check=True)

In [ ]:
import dvc.api
import pandas as pd

with dvc.api.open(
    "dvc_meta/df.parquet",   # จะใส่ path ของไฟล์จริง หรือ .dvc ก็ได้
    repo=".",
    remote="storage",
    mode="rb"                # 👈 สำคัญสำหรับ parquet
) as f:
    df = pd.read_parquet(f)  # ต้องมี pyarrow ติดตั้ง
print(df.head())

In [ ]:
import dvc.api
import pandas as pd

with dvc.api.open(
    "dvc_meta/newsseg_desc_not_null.csv",   # ใช้ path ของไฟล์จริงหรือ .dvc ก็ได้
    repo=".",
    remote="storage", 
    mode="r",            # text mode
    encoding="utf-8"     # ลอง "utf-8-sig" ถ้ายัง error
) as f:
    df = pd.read_csv(f)
print(df.head())

In [17]:
from src.etl.bronze.extract.data_structure_extract_strategy import DataExtractor

cfg_minio_csv = {
    "type": "minio",
    "endpoint": "localhost:9002",
    "access_key": "minio",
    "secret_key": "minio12345",
    "bucket": "test",
    "object": "test/myfile.csv",
    "secure": False,
    "encoding": "utf-8",
}

extractor_csv = DataExtractor.get_extractor(cfg=cfg_minio_csv)
df = extractor_csv.extractor()
df.head()


,title,description,categories
0,nba draft player survey: who's the most underr...,cooper flagg's energy and leadership are the b...,sport
1,£250million chelsea manor's owner says 'britai...,the luxury 300-year-old georgian manor belongi...,sport
2,'guardians of the galaxy' star says her oscar ...,oscar-winner zoe saldaña told people magazine ...,entertainment
3,exempting tips from taxes could hurt employees...,lawmakers recently passed measures to exempt t...,business
4,"what to expect in markets this week: tesla, go...",coming up: earnings reports are expected from ...,business


In [16]:
from minio import Minio
from io import BytesIO
from pathlib import Path
import pandas as pd

# สร้าง client
client = Minio(
    "localhost:9002",         # 👈 ใช้พอร์ต API ที่ map ไว้
    access_key="minio",
    secret_key="minio12345",
    secure=False,             # ถ้า http ให้ False, https ให้ True
)

bucket = "test"
object_name = "test/myfile.csv"  # ชื่อ object ต้องตรงเป๊ะ

# โหลด object
response = client.get_object(bucket, object_name)

try:
    data = response.read()  # โหลดทั้งหมดเข้าหน่วยความจำ
finally:
    response.close()
    response.release_conn()

# เลือกอ่านตามนามสกุลไฟล์
ext = Path(object_name).suffix.lower()
if ext == ".parquet":
    df = pd.read_parquet(BytesIO(data))  # ต้องติดตั้ง pyarrow
elif ext == ".csv":
    df = pd.read_csv(BytesIO(data), encoding="utf-8")
elif ext == ".json":
    df = pd.read_json(BytesIO(data))
else:
    raise ValueError(f"Unsupported file extension: {ext}")

print(df.head())


                                               title  \
0  nba draft player survey: who's the most underr...   
1  £250million chelsea manor's owner says 'britai...   
2  'guardians of the galaxy' star says her oscar ...   
3  exempting tips from taxes could hurt employees...   
4  what to expect in markets this week: tesla, go...   

                                         description     categories  
0  cooper flagg's energy and leadership are the b...          sport  
1  the luxury 300-year-old georgian manor belongi...          sport  
2  oscar-winner zoe saldaña told people magazine ...  entertainment  
3  lawmakers recently passed measures to exempt t...       business  
4  coming up: earnings reports are expected from ...       business  


In [14]:
from minio import Minio

client = Minio(
    "localhost:9002", access_key="minio", secret_key="minio12345", secure=False
)
# client.remove_bucket("test")
client.fput_object(
    "test",
    "test/myfile.csv",  # object name (ชื่อไฟล์ใน bucket)
    "C:/Users/Lenovo/Desktop/AI-News-Project/data/interim/data_news_segmentation/description_not_null/newsseg_desc_not_null_v20250811_144642.csv",  # path ไฟล์ในเครื่อง
)
print("Upload done")

Upload done
